<a href="https://colab.research.google.com/github/b3nn9/kaggle23/blob/main/EDA_aimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> 라이브러리 설치</h1>

In [ ]:
!pip install pandas scikit-learn optuna seaborn matplotlib catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainD = "/content/drive/MyDrive/aiming/train.csv"
testD = "/content/drive/MyDrive/aiming/test.csv"
train = pd.read_csv(trainD)
test = pd.read_csv(testD)

### EDA

###1. 데이터프레임의 기본 정보</br>
#### (1) shape</br>
형상(데이터의 행과 열 파악, 5**98개의 행(데이터 개수)와 2421개의 열(피쳐)**</br>
#### (2) info</br>
행과 열에 대한 간단한 정보 (개수와 첫-끝 피쳐명), 데이터타입 별 개수</br>
**object 4개 -> categorical**</br>
**나머지 -> numerical**</br>

In [ ]:
train.shape

(598, 2421)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Columns: 2421 entries, PRODUCT_ID to X_2870
dtypes: float64(2416), int64(1), object(4)
memory usage: 11.0+ MB


### 2. 데이터의 분포
#### (1) describe</br>
열마다 총 데이터 개수(null이 아닌 값)</br>
mean : 평균값</br>
std : 표준편차, 클수록 퍼진 형태</br>
25, 50, 75% : 사분위수에 해당하는 값</br>
min, max : 최대 최솟값</br>
1. 대략적으로 봤을 때, count값이 598(전체 데이터 수가 598이므로 not null)인 피쳐도 있고, 349인 피쳐도 여러 개 있고, 120인 피쳐도 여러 개 있다 </br>-> **null개수에 규칙성이 있다는 것을 추측할 수 있다** </br>
2. mean값이 0.xx대부터 몇만 이상까지 다양하다</br> -> **전처리 단계에서 정규화를 고려**</br>
3. min과 max값이 (피쳐 별 데이터의 분포) 전부 다르다</br>-> **max-min하여 데이터 분포 범위를 가지고 상관관계를 알아보아도 좋을 것 같다.**</br>
4. 사분위수</br>-> **IQR 이상치제거를 진행해도 좋을 것 같다. 또는 50%값과 평균의 차이를 알아보아도 좋을 것 같다.**


In [ ]:
train.describe()

Y_Class   Y_Quality         X_1         X_2         X_5         X_7  \
count  598.000000  598.000000  349.000000  349.000000  349.000000  349.000000   
mean     1.025084    0.530896    2.409742   95.123209   10.392550   48.802292   
std      0.565069    0.007401    5.895256    4.107640    0.489019    4.373824   
min      0.000000    0.500856    1.000000   87.000000   10.000000   45.000000   
25%      1.000000    0.527535    2.000000   93.000000   10.000000   45.000000   
50%      1.000000    0.530436    2.000000   95.000000   10.000000   45.000000   
75%      1.000000    0.533433    2.000000   98.000000   11.000000   51.000000   
max      2.000000    0.578841  103.000000  102.000000   11.000000   62.000000   

              X_8         X_9        X_11        X_12        X_13        X_15  \
count  349.000000  349.000000  349.000000  349.000000  349.000000  349.000000   
mean    10.048711   41.469914  497.050716  503.401433  488.307736    0.005731   
std      0.215571   10.515032   17.264680   17.975169   17.335538    0.107058   
min     10.000000   31.000000  433.900000  440.200000  424.300000    0.000000   
25%     10.000000   31.000000  487.400000  493.400000  478.500000    0.000000   
50%     10.000000   31.000000  499.900000  505.000000  489.800000    0.000000   
75%     10.000000   52.000000  509.600000  516.300000  500.900000    0.000000   
max     11.000000   52.000000  527.200000  538.000000  518.700000    2.000000   

             X_16        X_17        X_18        X_20        X_21        X_22  \
count  349.000000  349.000000  349.000000  349.000000  349.000000  349.000000   
mean   247.542120  248.068195  247.311748  497.982808  507.375358  492.303725   
std      0.052271    0.111636    0.076222   17.745316   18.512331   18.048610   
min    247.400000  247.700000  247.100000  469.000000  478.000000  461.000000   
25%    247.500000  248.000000  247.300000  485.000000  494.000000  479.000000   
50%    247.500000  248.100000  247.300000  495.000000  505.000000  490.000000   
75%    247.600000  248.100000  247.400000  507.000000  517.000000  501.000000   
max    247.700000  248.400000  247.500000  569.000000  583.000000  562.000000   

             X_24        X_38        X_39        X_40        X_41        X_42  \
count  349.000000  349.000000  349.000000  349.000000  349.000000  349.000000   
mean     0.191347    0.077364    1.982808   64.467049   65.559885   65.001146   
std      0.003419    0.001698    0.130173    1.450833    1.661704    1.155546   
min      0.190000    0.075000    1.000000   63.000000   63.500000   63.700000   
25%      0.190000    0.076000    2.000000   63.100000   64.100000   63.800000   
50%      0.190000    0.078000    2.000000   63.700000   65.100000   65.600000   
75%      0.190000    0.079000    2.000000   65.700000   66.800000   65.700000   
max      0.200000    0.079000    2.000000   67.300000   68.900000   67.500000   

             X_43        X_44        X_45        X_46        X_47        X_48  \
count  349.000000  349.000000  349.000000  349.000000  349.000000  349.000000   
mean    77.642120   70.273926   81.786246   71.000860   65.992837   75.460745   
std      1.219043    1.644839    0.428356    1.666824    1.814210    1.654489   
min     76.400000   68.300000   80.500000   68.700000   63.400000   73.500000   
25%     76.800000   68.600000   81.500000   69.800000   64.300000   73.600000   
50%     77.300000   69.000000   81.900000   70.700000   64.700000   74.700000   
75%     78.600000   72.000000   82.000000   72.200000   67.900000   76.900000   
max     82.000000   73.200000   82.400000   74.200000   68.100000   78.000000   

             X_49        X_50        X_51        X_52        X_53        X_54  \
count  349.000000  349.000000  349.000000  349.000000  349.000000  349.000000   
mean    77.290544   67.653009   79.652149   78.101719   66.722636   77.932951   
std      1.486509    1.983545    1.534162    1.644286    1.968875    1.415464   
min     75.600000   64.600000   77.4

#### (2) isnull, notnull</br>
데이터프레임 내에서 null인지 아닌지를 true/false 형태로 돌려줌</br>
모든 피쳐에 대해서 null 개수를 확인하기 위해서는 isnull과 sum()을 함께 사용하면 된다.</br>-> **598, 349, 78 등 여러가지 숫자가 공통적으로 나온다. 이 숫자들을 기준으로 칼럼을 모아 출력해서 파생변수를 생성하면 어떨까?**

In [ ]:
train.notnull().sum()

PRODUCT_ID      598
Y_Class         598
Y_Quality       598
TIMESTAMP       598
LINE            598
PRODUCT_CODE    598
X_1             349
X_2             349
X_5             349
X_7             349
X_8             349
X_9             349
X_11            349
X_12            349
X_13            349
X_15            349
X_16            349
X_17            349
X_18            349
X_20            349
X_21            349
X_22            349
X_24            349
X_38            349
X_39            349
X_40            349
X_41            349
X_42            349
X_43            349
X_44            349
X_45            349
X_46            349
X_47            349
X_48            349
X_49            349
X_50            349
X_51            349
X_52            349
X_53            349
X_54            349
X_55            349
X_56            349
X_57            349
X_58            349
X_59            349
X_60            349
X_61            349
X_62            349
X_63            349
X_64            349


In [ ]:
# Y_Quality에 따른 0/1/2 분류 기준 확인
# 0 = ~ 0.525067 / 1 = 0.5251086 ~ 0.534843 / 2 = 0.534951 ~ 

In [ ]:
#Y_Quality, PRODUCT_CODE 에 따른 0/1/2 분류 기준 확인

In [ ]:
#LINE, PRODUCT_CODE 에 따른 0/1/2 분류 기준 확인
# T0 ~ : A_31
# T1 ~ : T_31/O_31

In [ ]:
#Y_Quality, LINE, PRODUCT_CODE 에 따른 0/1/2 분류 기준 확인

In [ ]:
#LINE, PRODUCT_CODE 에 유효 변수 확인 X_ 

<h1> 변수 설명이 없기 때문에 칼럼별 Y분포를 통해서 변수를 예측 </h1>

<h3> 해당 칼럼의 Y class 별 갯수를 반환하는 함수 </h3>

<h3> 칼럼별 Y값 분포 출력 </h3>

<h2> 칼럼별 Y분포 종류 </h2>

<h2> Y 클래스 분포별로 칼럼 출력 </h2>

<h1> EDA이후 필요없는 Columns 제거</h1>

<h2> TrashC -> 모든 값이 Nan인 칼럼, Y 클래스값들이 1밖에 없는 칼럼, min과 max값이 동일한 칼럼</h2>

<h1> 결측치를 모두 평균으로 대체 </h1>

<h1> LINE, PRODUCT_CODE 칼럼들을 원핫인코딩으로 표현 </h1>

<h1> Regression or Classification에 따라 target 데이터셋 분류 </h1>

<h3> 데이터 최종 확인 </h3>

<h1> Regression모델 평가시 Class 값으로 변환 및 f1 Macro로 연산 </h1>

<h1> 모델 </h1>

<b> Optuna로 하이퍼파라미터 튜닝을 했지만 그냥 seed 값이 2023인 모델이 점수가 가장 높게 기록됨 </b>

<h2> 제출전 test예측결과 확인 </h2>